# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [121]:
import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.
# It comes with the tensorflow-datasets module, therefore, if you haven't please install the package using
# pip install tensorflow-datasets 
# or
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

## Data

That's where we load and preprocess our data.

In [122]:
# remember the comment from above
# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

# tfds.load actually loads a dataset (or downloads and then loads if that's the first time you use it) 
# in our case, we are interesteed in the MNIST; the name of the dataset is the only mandatory argument
# there are other arguments we can specify, which we can find useful
# mnist_dataset = tfds.load(name='mnist', as_supervised=True)
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# we will use this information a bit below and we will store it in mnist_info

# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 

In [123]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samlpes = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samlpes = tf.cast(num_validation_samlpes, tf.int64)

num_test_samlpes = 0.1 * mnist_info.splits['train'].num_examples
num_test_samlpes = tf.cast(num_test_samlpes, tf.int64)

def scale (image, label):
    image = tf.cast(image, tf.float32)
    image /= 522.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

Shuffeling

In [124]:
BUFFERS_SIZE = 10000
shuffeled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFERS_SIZE)
validation_data = shuffeled_train_and_validation_data.take(num_validation_samlpes)
train_data = shuffeled_train_and_validation_data.skip(num_validation_samlpes)



Batch size

In [125]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samlpes)

test_data = test_data.batch(num_test_samlpes)



In [126]:
validation_inputs, validation_targets = next(iter(validation_data))

Model

In [127]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])


choose an optimizer

In [128]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

training

In [129]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.2892 - accuracy: 0.9125 - val_loss: 0.1117 - val_accuracy: 0.9677 - 8s/epoch - 14ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1090 - accuracy: 0.9666 - val_loss: 0.0897 - val_accuracy: 0.9732 - 5s/epoch - 10ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0739 - accuracy: 0.9774 - val_loss: 0.0749 - val_accuracy: 0.9767 - 5s/epoch - 9ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0611 - accuracy: 0.9815 - val_loss: 0.0762 - val_accuracy: 0.9767 - 5s/epoch - 9ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0497 - accuracy: 0.9843 - val_loss: 0.0615 - val_accuracy: 0.9817 - 5s/epoch - 9ms/step


Test the model

In [130]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

2/2 [==============================] - 0s 151ms/step - loss: 0.0999 - accuracy: 0.9717
Test loss: 0.10. Test accuracy: 97.17%
